### QTM 350 Project
### Looking at the Global NOAA dataset

##### by Karoline Lin, Rachel Shanahan, John Cox, Erin O'Reilly, Hayley Roesler
##### on April 23, 2020

##### For our data science project, we decided to look at the NOAA-ids dataset found on AWS's open data registry. This dataset looks at 

In [99]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [100]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [101]:
def create_bucket(bucket):
    import logging

    try:
        s3.create_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        return 'Bucket ' + bucket + ' could not be created.'
    return 'Created or already exists ' + bucket + ' bucket.'

In [102]:
create_bucket('open-data-analytics-noaa-global')

'Created or already exists open-data-analytics-noaa-global bucket.'

In [103]:
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')

In [104]:
list_buckets(match='open')

Existing buckets containing "open" string:
  open-data-analytics-beat-aml
  open-data-analytics-nasa-nex
  open-data-analytics-new-afsis
  open-data-analytics-noaa-global
  open-data-analytics-nyc-tlc
  open-data-analytics-open-aq
  open-data-analytics-open-aq-fetches


In [105]:
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Matched file size is {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Bucket {bucket} total size is {total_size_gb/1024:3.1f}GB with {total_files} files')

In [106]:
#list_bucket_contents(bucket='noaa-global-hourly-pds', match='2019', size_mb=250)
#this command above lists the contents and data size of each bucket; we put this command in markdown for now as it produces a huge number of CSV files, from which we chose two 

In [107]:
def preview_csv_dataset(bucket, key, rows=10):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_csv(url, nrows=rows)
    return data

In [108]:
df = preview_csv_dataset(bucket='noaa-global-hourly-pds', key='2019/99999996409.csv', rows=100)

In [109]:
df1 = preview_csv_dataset(bucket='noaa-global-hourly-pds', key='2005/72220199999.csv', rows=100)

In [110]:
pd.set_option('display.max_columns', None)

In [111]:
df.head()

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AA1,AB1,AE1,AO1,CB1,CF1,CF2,CG1,CG2,CG3,CH1,CI1,CN1,CN2,CN3,CO1,CR1,CT1,CT2,CT3,CU1,CU2,CU3,CV1,CV2,CV3,CW1,GH1,IB2,KA1,KA2,KB1,KB2,KB3,KC1,KC2,KD1,KD2,KE1,KF1,OB1,EQD
0,99999996409,2019-01-01T00:00:00,I,68.6483,-149.3986,750.1,"TOOLIK LAKE 5 ENE, AK US",CRN05,99999,V020,"999,9,R,0069,1","99999,9,9,N","999999,9,9,9","-0057,1","+9999,9","99999,9","01,9999,9,0",NaN,NaN,"05,9999,9,0","05,+00000,1,0","0000,1,0","0681,1,0","+04700,9,0","+04840,9,0","+04690,9,0","05,-0058,1,0,0654,1,0","-0060,1,0,-0052,1,0,00003,1,0,00027,1,0","0135,1,0,0130,1,0,0116,1,0","-0046,1,0,-0062,1,0,00,1,0","014998,1,0,398630,1,0","99,-09","02514,1,0","-0057,1,0","-0057,1,0","-0057,1,0","+9999,9,0,0003,1,0","+9999,9,0,0003,1,0","+9999,9,0,0003,1,0","-0059,1,0,9999,9,0,-0050,1,0,9999,9,0","-0058,1,0,9999,9,0,-0050,1,0,9999,9,0","-0058,1,0,9999,9,0,-0050,1,0,9999,9,0","10190,1,0,29990,1,0","00000,1,0,00000,1,0,00000,1,0,00000,1,0","-0056,1,0,0002,1,0","010,M,-0050,1","010,N,-0058,1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-0054,1","060,0105,1,0,999,9,0,00103,1,0,99999,9,0",NaN
1,99999996409,2019-01-01T00:05:00,I,68.6483,-149.3986,750.1,"TOOLIK LAKE 5 ENE, AK US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","-0059,1","+9999,9","99999,9",NaN,NaN,NaN,"05,9999,9,0","05,+00000,1,0",NaN,NaN,"+04699,9,0","+04841,9,0","+04690,9,0","05,-0061,1,0,0659,1,0",NaN,NaN,NaN,NaN,"99,-09",NaN,"-0059,1,0","-0059,1,0","-0059,1,0",NaN,NaN,NaN,NaN,NaN,NaN,"10190,1,0,29990,1,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99999996409,2019-01-01T00:10:00,I,68.6483,-149.3986,750.1,"TOOLIK LAKE 5 ENE, AK US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","-0063,1","+9999,9","99999,9",NaN,NaN,NaN,"05,9999,9,0","05,+00000,1,0",NaN,NaN,"+04700,9,0","+04841,9,0","+04690,9,0","05,-0064,1,0,0687,1,0",NaN,NaN,NaN,NaN,"99,-09",NaN,"-0063,1,0","-0063,1,0","-0062,1,0",NaN,NaN,NaN,NaN,NaN,NaN,"10190,1,0,29990,1,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99999996409,2019-01-01T00:15:00,I,68.6483,-149.3986,750.1,"TOOLIK LAKE 5 ENE, AK US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","-0061,1","+9999,9","99999,9",NaN,NaN,NaN,"05,9999,9,0","05,+00000,1,0",NaN,NaN,"+04700,9,0","+04841,9,0","+04689,9,0","05,-0063,1,0,0672,1,0",NaN,NaN,NaN,NaN,"99,-09",NaN,"-0061,1,0","-0061,1,0","-0061,1,0",NaN,NaN,NaN,NaN,NaN,NaN,"10180,1,0,29990,1,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99999996409,2019-01-01T00:20:00,I,68.6483,-149.3986,750.1,"TOOLIK LAKE 5 ENE, AK US",CRN05,99999,V020,"999,9,9,9999,9","99999,9,9,N","999999,9,9,9","-0060,1","+9999,9","99999,9",NaN,NaN,NaN,"05,9999,9,0","05,+00000,1,0",NaN,NaN,"+04699,9,0","+04840,9,0","+04688,9,0","05,-0063,1,0,0677,1,0",NaN,NaN,NaN,NaN,"99,-09",NaN,"-0060,1,0","-0060,1,0","-0060,1,0",NaN,NaN,NaN,NaN,NaN,NaN,"10180,1,0,29930,1,0",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
df1.count()
## getting overview of each variable entry in our dataset

STATION            100
DATE               100
SOURCE             100
LATITUDE           100
LONGITUDE          100
ELEVATION          100
NAME               100
REPORT_TYPE        100
CALL_SIGN          100
QUALITY_CONTROL    100
WND                100
CIG                100
VIS                100
TMP                100
DEW                100
SLP                100
AA1                 11
AW1                 35
GA1                  4
GA2                  0
GA3                  0
GD1                 14
GD2                  0
GD3                  0
GF1                 97
MA1                100
MD1                  3
OC1                  0
REM                100
EQD                100
dtype: int64

In [113]:
columns = ['STATION', 'DATE','SOURCE','LATITUDE','LONGITUDE','TMP','ELEVATION','WND','NAME'] 
df_2019 = pd.DataFrame(df, columns=columns)
df_2005 = pd.DataFrame(df1,columns=columns)

In [114]:
df_join = df_2019.append(df_2005)
#df_join
df_join.head()
#join the datasets together

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,TMP,ELEVATION,WND,NAME
0,99999996409,2019-01-01T00:00:00,I,68.6483,-149.3986,"-0057,1",750.1,"999,9,R,0069,1","TOOLIK LAKE 5 ENE, AK US"
1,99999996409,2019-01-01T00:05:00,I,68.6483,-149.3986,"-0059,1",750.1,"999,9,9,9999,9","TOOLIK LAKE 5 ENE, AK US"
2,99999996409,2019-01-01T00:10:00,I,68.6483,-149.3986,"-0063,1",750.1,"999,9,9,9999,9","TOOLIK LAKE 5 ENE, AK US"
3,99999996409,2019-01-01T00:15:00,I,68.6483,-149.3986,"-0061,1",750.1,"999,9,9,9999,9","TOOLIK LAKE 5 ENE, AK US"
4,99999996409,2019-01-01T00:20:00,I,68.6483,-149.3986,"-0060,1",750.1,"999,9,9,9999,9","TOOLIK LAKE 5 ENE, AK US"


In [115]:
df_join.groupby(['STATION']).mean()
#observes the coordinates and elevation of both stations that we are studying

,LATITUDE,LONGITUDE,ELEVATION
STATION,,,
72220199999,35.5840,-79.1010,75.0
99999996409,68.6483,-149.3986,750.1


In [116]:
df_join.groupby(['TMP']).mean()

,STATION,LATITUDE,LONGITUDE,ELEVATION
TMP,,,,
"+0020,1",72220199999,35.5840,-79.1010,75.0
"+0030,1",72220199999,35.5840,-79.1010,75.0
"+0040,1",72220199999,35.5840,-79.1010,75.0
"+0050,1",72220199999,35.5840,-79.1010,75.0
"+0060,1",72220199999,35.5840,-79.1010,75.0
"+0070,1",72220199999,35.5840,-79.1010,75.0
"+0080,1",72220199999,35.5840,-79.1010,75.0
"+0090,1",72220199999,35.5840,-79.1010,75.0
"+0100,1",72220199999,35.5840,-79.1010,75.0
